In [ ]:
#BEGIN

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls

Mounted at /content/drive
drive  sample_data


In [87]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
# General
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import itertools
from pandas import DataFrame

# Visualization
import matplotlib.font_manager as fm
from matplotlib.collections import QuadMesh
import seaborn as sn
import plotly.express as px

# Feature extraction approach
from sklearn.feature_extraction.text import TfidfVectorizer
from string import punctuation
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

# Classification
import xgboost as xgb
import lightgbm as lgbm
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from simpletransformers.classification import ClassificationModel
import torch

# Parallelize apply on Pandas
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB

import nltk
from nltk.util import ngrams
from collections import Counter
import heapq

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [89]:
# path  = '/content/drive/My Drive/CS_635/temp/'
path  = '/content/gdrive/MyDrive/'

In [90]:
# train = pd.read_csv(path+'train_att1.csv')
# test = pd.read_csv(path+'test_att1.csv')
# val = pd.read_csv(path+'val_att1.csv')

In [91]:
# train = train.drop(['Unnamed: 0'], axis=1)
# test = test.drop(['Unnamed: 0'], axis=1)
# val = val.drop(['Unnamed: 0'], axis=1)

In [92]:
# def label_change(data):
#     for i in range(len(data)):
#         if data['label'][i] == 'HUMAN':
#             data['label'][i] = 0
#         elif data['label'][i] == 'GROVER':
#             data['label'][i] = 1
#         elif data['label'][i] == 'GPT2':
#             data['label'][i] = 2
#         elif data['label'][i] == 'GPT-j-6B':
#             data['label'][i] = 3
#     return data

# import re
# def clean_df(df):
#     for i in range(len(df)):
#         df['text'].loc[i] = re.sub('\s+', ' ', df['text'].iloc[i])
#     return df



In [93]:
# def label_change(data):
#     for i in range(len(data)):
#         if data['label'][i] == 'HUMAN':
#             data['label'][i] = 0
#         elif data['label'][i] == 'GROVER':
#             data['label'][i] = 1
#         elif data['label'][i] == 'GPT2':
#             data['label'][i] = 2
#         elif data['label'][i] == 'GPT-j-6B':
#             data['label'][i] = 3
#     return data

In [94]:
drive.mount('/content/gdrive/', force_remount=True)

data_dir = '/content/gdrive/MyDrive'
filename = 'p2_train.csv'
test_filename = 'p2_test.csv'
unclean_data_filename = 'unclean.csv'
!cp $data_dir/$filename .

Mounted at /content/gdrive/


In [95]:
# data = pd.read_csv(path+'p2_dataset.csv')
data1 = pd.read_csv(f'{data_dir}/{filename}')
data2 = pd.read_csv(f'{data_dir}/{test_filename}')

In [96]:
data3 = pd.read_csv(f'{data_dir}/{unclean_data_filename}')

In [97]:
data3

,Unnamed: 0,Generation,label
0,0,latest headlines on cnn business - google's go...,gpt2
1,1,china wants to take a victory lap over its han...,gpt2
2,2,coronavirus disinformation creates challenges ...,gpt2
3,3,china coronavirus: eating wild animals made il...,gpt2
4,4,china's economy could shrink for the first tim...,gpt2
...,...,...,...
11721,1061,\nAll of it.,instructgpt
11722,1062,"\nTo keep a space station clean, astronauts mu...",instructgpt
11723,1063,\nMost cities require tenants to pay the first...,instructgpt
11724,1064,. It’s easy to browse and customise the app to...,instructgpt


In [98]:
import re
# from cleantext import clean

def clean_individual_texts(data):
    gen_text = []

#     for i in data['Generation']:
#         words = ''.join([str(elem) for elem in i]) 
#         words = words.split()
#         gen_text.append("".join(eval(i)))
        
    new_gen = []
    for i in data['Generation']:
#         ans = re.sub('[^A-Za-z0-9]+', " ", str(i)) 
        ans = re.sub('<unk>', " ", str(i))
        ans = re.sub("\'\'", ' ', str(ans))
        ans = re.sub("\n", ' ', str(ans))
        ans = re.sub(" \ ", ' ', str(ans) )
        ans = re.sub('<UNK>', " ", str(ans))
        ans = re.sub('News', " ", str(ans))
        ans = re.sub('<|endoftext|>', " ", str(ans))
        ans = re.sub('<eos>', " ", str(ans))
        ans = re.sub('<eod>', " ", str(ans))
        ans = re.sub('<eop>', " ", str(ans))
#         ans = re.sub('CNN', " ", str(ans))
        
        new_gen.append(ans.replace('\n', ' ').replace('\r', ''))

    return pd.DataFrame({'Generation': new_gen, 'label': data['label']})

data4 = clean_individual_texts(data3)

In [99]:
data4

,Generation,label
0,latest headlines on cnn business - google's go...,gpt2
1,china wants to take a victory lap over its han...,gpt2
2,coronavirus disinformation creates challenges ...,gpt2
3,china coronavirus: eating wild animals made il...,gpt2
4,china's economy could shrink for the first tim...,gpt2
...,...,...
11721,All of it.,instructgpt
11722,"To keep a space station clean, astronauts mus...",instructgpt
11723,Most cities require tenants to pay the first ...,instructgpt
11724,. It’s easy to browse and customise the app to...,instructgpt


In [100]:
from sklearn.metrics import classification_report, roc_curve, auc, f1_score, accuracy_score, confusion_matrix, recall_score, precision_score
    
def get_classification_report(y_test, y_pred, label_count):
    matrix = confusion_matrix(y_test, y_pred, labels = list(range(0,label_count)))
    mat = matrix.diagonal()/matrix.sum(axis=1)
    print(classification_report(y_test, y_pred, labels = list(range(0,label_count)),
                                digits=4))
    print('confusion matrix: ', mat)

    Accuracy = accuracy_score(y_test,y_pred)
    F1 = f1_score(y_test, y_pred, average='macro')
    print("Accuracy:", Accuracy)

    rec = recall_score(y_test, y_pred, average='macro')
    print('Recall: ', rec)
    prec = precision_score(y_test, y_pred, average='macro')
    print('Precision: ', prec)

    print('F1:', F1)

In [101]:
# train = label_change(train)
# test = label_change(test)
# val = label_change(val)

# train = clean_df(train)
# test = clean_df(test)
# val = clean_df(val)

In [102]:
# Text pre-processing as required to compare the content of different authors
def fil_sent(sent):
    """
    Filter stopwords
    """
    filtered_sentence = ' '.join([w for w in sent.split() if not w in stop_words])
    filtered_sentence = ''.join([w for w in filtered_sentence if w not in list(punctuation)])
    filtered_sentence = filtered_sentence.strip()
    filtered_sentence = filtered_sentence.split()
    return filtered_sentence

def process(sent):
    """
    Apply stemming
    """
    sent = str(sent)
    return fil_sent(' '.join([ps.stem(str(x).lower()) for x in word_tokenize(sent)]))

stop_words = set(stopwords.words('english'))
ps = PorterStemmer() 

In [103]:
# Similarity between two authors' content
def jaccard_similarity(list1, list2):

    list1 = list1[:512]
    list2 = list2[:512]
    
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection

    if union > 0:
      return float(intersection) / union
    else:
      return 0

In [104]:
# Computes the relative hardness
def rel_hardness(df, col_id, col_text):

  n_labels = len(list(set(df[col_text].values)))
  return 1/(n_labels * (n_labels - 1) * 0.5) * sum([jaccard_similarity(x[0], x[1]) for x in itertools.combinations(all_text_all_category(df, col_id, col_text), 2) if x[0] != x[1]])

In [105]:
def fil_sent(sent):
  """
  Filter stopwords
  """

  filtered_sentence = ' '.join([w for w in sent.split() if not w in stop_words])
  return filtered_sentence

def process(sent):
  """
  Apply stemming
  """

  sent = str(sent)
  return fil_sent(' '.join([ps.stem(str(x).lower()) for x in word_tokenize(sent)]))

stop_words = set(stopwords.words('english'))
ps = PorterStemmer() 

def extract_style(text):
    """
    Extracting stylometric features of a text
    """

    text = str(text)
    len_text = len(text)
    len_words = len(text.split())
    avg_len = np.mean([len(t) for t in text.split()])
    num_short_w = len([t for t in text.split() if len(t) < 3])
    per_digit = sum(t.isdigit() for t in text)/len(text)
    per_cap = sum(1 for t in text if t.isupper())/len(text)
    f_a = sum(1 for t in text if t.lower() == "a")/len(text)
    f_b = sum(1 for t in text if t.lower() == "b")/len(text)
    f_c = sum(1 for t in text if t.lower() == "c")/len(text)
    f_d = sum(1 for t in text if t.lower() == "d")/len(text)
    f_e = sum(1 for t in text if t.lower() == "e")/len(text)
    f_f = sum(1 for t in text if t.lower() == "f")/len(text)
    f_g = sum(1 for t in text if t.lower() == "g")/len(text)
    f_h = sum(1 for t in text if t.lower() == "h")/len(text)
    f_i = sum(1 for t in text if t.lower() == "i")/len(text)
    f_j = sum(1 for t in text if t.lower() == "j")/len(text)
    f_k = sum(1 for t in text if t.lower() == "k")/len(text)
    f_l = sum(1 for t in text if t.lower() == "l")/len(text)
    f_m = sum(1 for t in text if t.lower() == "m")/len(text)
    f_n = sum(1 for t in text if t.lower() == "n")/len(text)
    f_o = sum(1 for t in text if t.lower() == "o")/len(text)
    f_p = sum(1 for t in text if t.lower() == "p")/len(text)
    f_q = sum(1 for t in text if t.lower() == "q")/len(text)
    f_r = sum(1 for t in text if t.lower() == "r")/len(text)
    f_s = sum(1 for t in text if t.lower() == "s")/len(text)
    f_t = sum(1 for t in text if t.lower() == "t")/len(text)
    f_u = sum(1 for t in text if t.lower() == "u")/len(text)
    f_v = sum(1 for t in text if t.lower() == "v")/len(text)
    f_w = sum(1 for t in text if t.lower() == "w")/len(text)
    f_x = sum(1 for t in text if t.lower() == "x")/len(text)
    f_y = sum(1 for t in text if t.lower() == "y")/len(text)
    f_z = sum(1 for t in text if t.lower() == "z")/len(text)
    f_1 = sum(1 for t in text if t.lower() == "1")/len(text)
    f_2 = sum(1 for t in text if t.lower() == "2")/len(text)
    f_3 = sum(1 for t in text if t.lower() == "3")/len(text)
    f_4 = sum(1 for t in text if t.lower() == "4")/len(text)
    f_5 = sum(1 for t in text if t.lower() == "5")/len(text)
    f_6 = sum(1 for t in text if t.lower() == "6")/len(text)
    f_7 = sum(1 for t in text if t.lower() == "7")/len(text)
    f_8 = sum(1 for t in text if t.lower() == "8")/len(text)
    f_9 = sum(1 for t in text if t.lower() == "9")/len(text)
    f_0 = sum(1 for t in text if t.lower() == "0")/len(text)
    f_e_0 = sum(1 for t in text if t.lower() == "!")/len(text)
    f_e_1 = sum(1 for t in text if t.lower() == "-")/len(text)
    f_e_2 = sum(1 for t in text if t.lower() == ":")/len(text)
    f_e_3 = sum(1 for t in text if t.lower() == "?")/len(text)
    f_e_4 = sum(1 for t in text if t.lower() == ".")/len(text)
    f_e_5 = sum(1 for t in text if t.lower() == ",")/len(text)
    f_e_6 = sum(1 for t in text if t.lower() == ";")/len(text)
    f_e_7 = sum(1 for t in text if t.lower() == "'")/len(text)
    f_e_8 = sum(1 for t in text if t.lower() == "/")/len(text)
    f_e_9 = sum(1 for t in text if t.lower() == "(")/len(text)
    f_e_10 = sum(1 for t in text if t.lower() == ")")/len(text)
    f_e_11 = sum(1 for t in text if t.lower() == "&")/len(text)
    richness = len(list(set(text.split())))/len(text.split())

    return pd.Series([avg_len, len_text, len_words, num_short_w, per_digit, per_cap, f_a, f_b, f_c, f_d, f_e, f_f, f_g, f_h, f_i, f_j, f_k, f_l, f_m, f_n, f_o, f_p, f_q, f_r, f_s, f_t, f_u, f_v, f_w, f_x, f_y, f_z, f_0, f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8, f_9, f_e_0, f_e_1, f_e_2, f_e_3, f_e_4, f_e_5, f_e_6, f_e_7, f_e_8, f_e_9, f_e_10, f_e_11, richness])

def return_best_bi_grams(text):
  bigrams = ngrams(text,2)

  data = dict(Counter(bigrams))
  list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
  return list_ngrams

def return_best_tri_grams(text):
  trigrams = ngrams(text,3)

  data = dict(Counter(trigrams))
  list_ngrams = heapq.nlargest(100, data.keys(), key=lambda k: data[k])
  return list_ngrams

def find_freq_n_gram_in_txt(text, list_bigram, list_trigram):

  to_ret = []

  num_bigrams = len(Counter(zip(text,text[1:])))
  num_trigrams = len(Counter(zip(text,text[1:],text[2:])))

  for n_gram in list_bigram: 
      to_ret.append(text.count(''.join(n_gram))/num_bigrams)

  for n_gram in list_trigram: 
      to_ret.append(text.count(''.join(n_gram))/num_trigrams)

  return to_ret

In [106]:
import nltk
nltk.download('universal_tagset')
nltk.download('averaged_perceptron_tagger')
from nltk import word_tokenize
def extract_pos(text):
  text = str(text)
  tokens = word_tokenize(text)
  tags = nltk.pos_tag(tokens, tagset = "universal")
  CC = sum(1 for word, tag in tags if tag == 'CC')/len(text),
  CD = sum(1 for word, tag in tags if tag == 'CD')/len(text),
  DT = sum(1 for word, tag in tags if tag == 'DT')/len(text),
  EX = sum(1 for word, tag in tags if tag == 'EX')/len(text),
  FW = sum(1 for word, tag in tags if tag == 'FW')/len(text),
  IN = sum(1 for word, tag in tags if tag == 'IN')/len(text),
  JJ = sum(1 for word, tag in tags if tag == 'JJ')/len(text),
  JJR = sum(1 for word, tag in tags if tag == 'JJR')/len(text),
  JJS = sum(1 for word, tag in tags if tag == 'JJS')/len(text),
  LS = sum(1 for word, tag in tags if tag == 'LS')/len(text),
  MD = sum(1 for word, tag in tags if tag == 'MD')/len(text),
  NN = sum(1 for word, tag in tags if tag == 'NN')/len(text),
  NNS = sum(1 for word, tag in tags if tag == 'NNS')/len(text),
  NNP = sum(1 for word, tag in tags if tag == 'NNP')/len(text),
  PDT = sum(1 for word, tag in tags if tag == 'PDT')/len(text),
  POS = sum(1 for word, tag in tags if tag == 'POS')/len(text),
  PRP = sum(1 for word, tag in tags if tag == 'PRP')/len(text),
  PRPs = sum(1 for word, tag in tags if tag == 'PRP$')/len(text),
  RB = sum(1 for word, tag in tags if tag == 'RB')/len(text),
  RBR = sum(1 for word, tag in tags if tag == 'RBR')/len(text),
  RBS = sum(1 for word, tag in tags if tag == 'RBS')/len(text),
  RP = sum(1 for word, tag in tags if tag == 'RP')/len(text),
  SYM = sum(1 for word, tag in tags if tag == 'SYM')/len(text),
  TO = sum(1 for word, tag in tags if tag == 'TO')/len(text),
  UH = sum(1 for word, tag in tags if tag == 'UH')/len(text),
  VB = sum(1 for word, tag in tags if tag == 'VB')/len(text),
  VBD = sum(1 for word, tag in tags if tag == 'VBD')/len(text),
  VBG = sum(1 for word, tag in tags if tag == 'VBG')/len(text),
  VBN = sum(1 for word, tag in tags if tag == 'VBN')/len(text),
  VBP = sum(1 for word, tag in tags if tag == 'VBP')/len(text),
  VBZ = sum(1 for word, tag in tags if tag == 'VBZ')/len(text),
  WDT = sum(1 for word, tag in tags if tag == 'WDT')/len(text),
  WP = sum(1 for word, tag in tags if tag == 'WP')/len(text),
  WPs = sum(1 for word, tag in tags if tag == 'WP$')/len(text),
  WRB = sum(1 for word, tag in tags if tag == 'WRB')/len(text),

  return pd.Series([CC[0], CD[0], DT[0], EX[0], FW[0], IN[0], JJ[0], JJR[0], JJS[0], LS[0], MD[0], NN[0], NNS[0], NNP[0], PDT[0], POS[0], PRP[0], PRPs[0], RB[0], RBR[0], RBS[0], RP[0], SYM[0], TO[0], UH[0], VB[0], VBD[0], VBG[0], VBN[0], VBP[0], VBZ[0], WDT[0], WP[0], WPs[0], WRB[0]])


  # return pd.Series([CC, CD, DT, EX, FW, IN, JJ, JJR, JJS, LS, MD, NN, NNS, NNP, PDT, POS, PRP, PRPs, RB, RBR, RBS, RP, SYM, TO, UH, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WPs, WRB])



[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [107]:
# import nltk
# from collections import Counter
# nltk.download('averaged_perceptron_tagger')
# nltk.download('universal_tagset')
# from nltk import word_tokenize
# text = 'This is a sentence'
# text = str(text)
# tokens = word_tokenize(text)
# tags = nltk.pos_tag(tokens, tagset = "universal")
# d = Counter(tags)


In [108]:
# len(tags)

In [109]:
# df = pd.concat([train, test, val])
df = pd.concat([data1, data2])
df['text_tfidf'] = df['text'].parallel_apply(lambda x: process(x))
df[["avg_len", "len_text", "len_words", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9",  "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness"]] = df['text'].parallel_apply(lambda x: extract_style(x))
# df[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']] = df['text'].parallel_apply(lambda x: extract_pos(x))

In [110]:
df[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']] = df['text'].parallel_apply(lambda x: extract_pos(x))

In [111]:
## Heading

In [112]:
df

,Unnamed: 0,text,label,text_tfidf,avg_len,len_text,len_words,num_short_w,per_digit,per_cap,...,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WPs,WRB
0,7744,longlost pictur archiv celebr glamour seneg wr...,0,longlost pictur archiv celebr glamour seneg wr...,5.448113,2733.0,424.0,8.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10536,advanc understand natur world respons current ...,1,advanc understand natur world respon current s...,6.468750,238.0,32.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7043,itali test western economi bear almost total s...,1,itali test western economi bear almost total s...,5.820961,1561.0,229.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5930,south africa box granni juke jab way healthier...,1,south africa box granni juke jab way healthier...,5.508333,2342.0,360.0,17.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,11660,coronaviru pandem nation tragedi hundr thousan...,1,coronaviru pandem nation tragedi hundr thousan...,5.690476,842.0,126.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,11701,suggest polit affili play role afraid peopl co...,1,suggest polit affili play role afraid peopl co...,5.738255,1003.0,149.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
422,11703,stephen sondheim american compos lyricist one ...,1,stephen sondheim american compo lyricist one i...,6.000000,545.0,78.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
423,11708,ahm janko survivor brutal boko haram attack no...,1,ahm janko survivor brutal boko haram attack no...,5.452174,741.0,115.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
424,11709,step invest contact trace expand test today us...,1,step invest contact trace expand test today us...,5.257919,1382.0,221.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
df = df.dropna()
df.shape

(2131, 94)

In [114]:
text = ' '.join(df['text'].values)
list_bigram = return_best_bi_grams(text)
list_trigram = return_best_tri_grams(text)

In [115]:
ind = train_test_split(df[['text', 'label']], test_size=0.2, stratify=df['label'])
ind_train = list(ind[0].index)
ind_test = list(ind[1].index)

In [116]:
nlp_train = df.loc[ind_train]
nlp_test = df.loc[ind_test]

In [117]:
y_train = nlp_train['label']
y_test = nlp_test['label']

In [83]:
# Training TF-IDF

In [118]:
vectorize = TfidfVectorizer()
X_train = vectorize.fit_transform(nlp_train['text_tfidf'])
X_test = vectorize.transform(nlp_test['text_tfidf'])

clf = LogisticRegression(random_state=0).fit(X_train, nlp_train['label'].astype('int'))
y_pred = clf.predict(X_test)

score_lr = accuracy_score(nlp_test['label'].astype('int'), y_pred)
f1_lr = f1_score(nlp_test['label'].astype('int'), y_pred, average="macro")

# plot_confusion_matrix_from_data(nlp_test['label'].astype('int'), y_pred)

print("Training done, accuracy is : ", score_lr)
print("Training done, f1-score is : ", f1_lr)
confusion_matrix(nlp_test['label'].astype('int'), y_pred)

print(get_classification_report(y_test, y_pred, 2))

Training done, accuracy is :  0.9
Training done, f1-score is :  0.8999599839935974
              precision    recall  f1-score   support

           0     0.8734    0.9324    0.9020       296
           1     0.9296    0.8684    0.8980       304

    accuracy                         0.9000       600
   macro avg     0.9015    0.9004    0.9000       600
weighted avg     0.9019    0.9000    0.8999       600

confusion matrix:  [0.93243243 0.86842105]
Accuracy: 0.9
Recall:  0.9004267425320057
Precision:  0.9014975931538598
F1: 0.8999599839935974
None


In [119]:
# def plot_confusion_matrix_from_data(y_test, predictions, columns=None, annot=True, cmap="Oranges",
#       fmt='.2f', fz=11, lw=0.5, cbar=False, figsize=[8,8], show_null_values=0, pred_val_axis='lin'):
#     """
#         plot confusion matrix function with y_test (actual values) and predictions (predic),
#         whitout a confusion matrix yet
#     """
#     from sklearn.metrics import confusion_matrix
#     from pandas import DataFrame

#     #data
#     if(not columns):
#         #labels axis integer:
#         ##columns = range(1, len(np.unique(y_test))+1)
#         #labels axis string:
#         from string import ascii_uppercase
#         columns = ['class %s' %(i) for i in list(ascii_uppercase)[0:len(np.unique(y_test))]]

In [33]:
#Training Random Forest

In [120]:
clf = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(X_train, nlp_train['label'].astype('int'))
y_pred = clf.predict(X_test)

score_lr = accuracy_score(nlp_test['label'].astype('int'), y_pred)
f1_lr = f1_score(nlp_test['label'].astype('int'), y_pred, average="macro")

# plot_confusion_matrix_from_data(nlp_test['label'].astype('int'), y_pred)

print("Training done, accuracy is : ", score_lr)
print("Training done, f1-score is : ", f1_lr)
confusion_matrix(nlp_test['label'].astype('int'), y_pred)

print(get_classification_report(y_test, y_pred, 2))

Training done, accuracy is :  0.955
Training done, f1-score is :  0.954993874166206
              precision    recall  f1-score   support

           0     0.9529    0.9561    0.9545       296
           1     0.9571    0.9539    0.9555       304

    accuracy                         0.9550       600
   macro avg     0.9550    0.9550    0.9550       600
weighted avg     0.9550    0.9550    0.9550       600

confusion matrix:  [0.95608108 0.95394737]
Accuracy: 0.955
Recall:  0.9550142247510669
Precision:  0.954978831216455
F1: 0.954993874166206
None


In [121]:
clf = xgb.XGBClassifier().fit(X_train, nlp_train['label'].astype('int'))
y_pred = clf.predict(X_test)

score_lr = accuracy_score(nlp_test['label'].astype('int'), y_pred)
f1_lr = f1_score(nlp_test['label'].astype('int'), y_pred, average="macro")

# plot_confusion_matrix_from_data(nlp_test['label'].astype('int'), y_pred)

print("Training done, accuracy is : ", score_lr)
print("Training done, f1-score is : ", f1_lr)
# confusion_matrix(nlp_test['label'].astype('int'), y_pred)

print(get_classification_report(y_test, y_pred, 2))

Training done, accuracy is :  0.9583333333333334
Training done, f1-score is :  0.9582822290639367
              precision    recall  f1-score   support

           0     0.9788    0.9358    0.9568       296
           1     0.9401    0.9803    0.9597       304

    accuracy                         0.9583       600
   macro avg     0.9594    0.9580    0.9583       600
weighted avg     0.9592    0.9583    0.9583       600

confusion matrix:  [0.93581081 0.98026316]
Accuracy: 0.9583333333333334
Recall:  0.9580369843527738
Precision:  0.959430839027544
F1: 0.9582822290639367
None


In [122]:
from sklearn.svm import SVC
clf = SVC().fit(X_train, nlp_train['label'].astype('int'))
y_pred = clf.predict(X_test)

score_lr = accuracy_score(nlp_test['label'].astype('int'), y_pred)
f1_lr = f1_score(nlp_test['label'].astype('int'), y_pred, average="macro")

# plot_confusion_matrix_from_data(nlp_test['label'].astype('int'), y_pred)

print("Training done, accuracy is : ", score_lr)
print("Training done, f1-score is : ", f1_lr)
confusion_matrix(nlp_test['label'].astype('int'), y_pred)

print(get_classification_report(y_test, y_pred, 2))

Training done, accuracy is :  0.915
Training done, f1-score is :  0.9149940968122787
              precision    recall  f1-score   support

           0     0.8964    0.9358    0.9157       296
           1     0.9347    0.8947    0.9143       304

    accuracy                         0.9150       600
   macro avg     0.9156    0.9153    0.9150       600
weighted avg     0.9158    0.9150    0.9150       600

confusion matrix:  [0.93581081 0.89473684]
Accuracy: 0.915
Recall:  0.915273826458037
Precision:  0.9155740166149535
F1: 0.9149940968122787
None


In [123]:
from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB().fit(X_train, nlp_train['label'].astype('int'))
y_pred = clf.predict(X_test)

score_lr = accuracy_score(nlp_test['label'].astype('int'), y_pred)
f1_lr = f1_score(nlp_test['label'].astype('int'), y_pred, average="macro")

# plot_confusion_matrix_from_data(nlp_test['label'].astype('int'), y_pred)

print("Training done, accuracy is : ", score_lr)
print("Training done, f1-score is : ", f1_lr)
confusion_matrix(nlp_test['label'].astype('int'), y_pred)

print(get_classification_report(y_test, y_pred, 2))

Training done, accuracy is :  0.745
Training done, f1-score is :  0.7334301270417423
              precision    recall  f1-score   support

           0     0.8994    0.5439    0.6779       296
           1     0.6793    0.9408    0.7890       304

    accuracy                         0.7450       600
   macro avg     0.7894    0.7424    0.7334       600
weighted avg     0.7879    0.7450    0.7342       600

confusion matrix:  [0.54391892 0.94078947]
Accuracy: 0.745
Recall:  0.7423541963015647
Precision:  0.7893881288233655
F1: 0.7334301270417423
None


In [42]:
# Training BERT

In [43]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=2, args={'reprocess_input_data': True, 'overwrite_output_dir': True,  'num_train_epochs' : 15}, use_cuda=True) 
model.train_model(nlp_train[['text', 'label']])
predictions, raw_outputs = model.predict(list(nlp_test['text']))
score_bert = accuracy_score(predictions, nlp_test['label'])
f1_bert = f1_score(predictions, nlp_test['label'], average="macro")
predictions, raw_out_train = model.predict(list(nlp_train['text']))
print("Training done, accuracy is : ", score_bert)
print("Training done, f1-score is : ", f1_bert)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/2379 [00:00<?, ?it/s]

Epoch:   0%|          | 0/15 [00:00<?, ?it/s]

Running Epoch 0 of 15:   0%|          | 0/298 [00:00<?, ?it/s]

...

  0%|          | 0/604 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2379 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

Training done, accuracy is :  0.9403973509933775
Training done, f1-score is :  0.9403738153738154


In [126]:
predictions, raw_outputs = model.predict(list(nlp_test['text']))
score_bert = accuracy_score(predictions, nlp_test['label'])
precision_bert = precision_score(predictions, nlp_test['label'])
recall_bert = recall_score(predictions, nlp_test['label'])
f1_bert = f1_score(predictions, nlp_test['label'], average="macro")
predictions, raw_out_train = model.predict(list(nlp_train['text']))
print("Training done, accuracy is : ", score_bert)
print("Training done, f1-score is : ", f1_bert)
print("Training done, Precision-core is : ", precision_bert)
print("Training done, Recall-score is : ", recall_bert)

  0%|          | 0/600 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

  0%|          | 0/2383 [00:00<?, ?it/s]

  0%|          | 0/298 [00:00<?, ?it/s]

Training done, accuracy is :  0.99
Training done, f1-score is :  0.9899995555358017
Training done, Precision-core is :  0.9835526315789473
Training done, Recall-score is :  0.9966666666666667


In [ ]:
#XGBOOST

In [135]:
#Training Style Classifier
print("Training Style Classifier")

X_style_train = nlp_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB",  ]]
X_style_test = nlp_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB", ]]

clf = xgb.XGBClassifier().fit(X_style_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_style_test)
y_proba = clf.predict_proba(X_style_test)
y_proba_train = clf.predict_proba(X_style_train)
score_style = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_style)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))
print(clf.feature_importances_)
print("\nTraining pos Classifier")

#Training pos Classifier

X_pos_train = nlp_train[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]
X_pos_test = nlp_test[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]

clf = xgb.XGBClassifier().fit(X_pos_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_pos_test)
y_proba_pos = clf.predict_proba(X_pos_test)
y_proba_train_pos = clf.predict_proba(X_pos_train)
score_pos = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_pos)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))
print(clf.feature_importances_)
print("\nModel Combination")

feat_for_BERT_LR_train = np.concatenate([raw_out_train, y_proba_train], axis=1)
feat_for_BERT_LR_test = np.concatenate([raw_outputs, y_proba], axis=1)

# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['label'])
clf = xgb.XGBClassifier().fit(feat_for_BERT_LR_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_LR_test)
score_comb = accuracy_score(nlp_test['label'], y_pred)
f1_comb = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_comb)
print("Training done, f1-score is : ", f1_comb)
confusion_matrix(nlp_test['label'], y_pred)

# plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

print(get_classification_report(y_test, y_pred, 2))
print(clf.feature_importances_)
print("\nChar N-gram")

#char n-gram
feats_train = nlp_train['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values
feats_test = nlp_test['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values

feats_train = pd.DataFrame(feats_train)[0].apply(lambda x: pd.Series(x))
feats_test = pd.DataFrame(feats_test)[0].apply(lambda x: pd.Series(x))

clf_char = xgb.XGBClassifier().fit(feats_train, nlp_train['label'])
# clf_char = LogisticRegression(random_state=0).fit(feats_train, nlp_train['label'])
y_pred = clf_char.predict(feats_test)
y_proba = clf_char.predict_proba(feats_test)
y_proba_train = clf_char.predict_proba(feats_train)

score_char = accuracy_score(nlp_test['label'], y_pred)
f1_char = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_char)
print("Training done, f1-score is : ", f1_char)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))
print(clf.feature_importances_)
print("\nfEATURES FROM bert & Stylometric")

feat_for_BERT_full_train = np.concatenate([feat_for_BERT_LR_train, y_proba_train], axis=1)
feat_for_BERT_full_test = np.concatenate([feat_for_BERT_LR_test, y_proba], axis=1)

clf = xgb.XGBClassifier().fit(feat_for_BERT_full_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_full_test)
score_comb_fin = accuracy_score(nlp_test['label'], y_pred)
f1_comb_fin = f1_score(nlp_test['label'], y_pred, average="macro")
print("Training done, accuracy is : ", score_comb_fin)
print("Training done, f1-score is : ", f1_comb_fin)
confusion_matrix(nlp_test['label'], y_pred)

list_scores, list_f1 = [] , []
list_scores.append([score_lr, score_bert, score_style, score_comb])
list_f1.append([f1_lr, f1_bert, f1_style, f1_comb])

print(get_classification_report(y_test, y_pred, 2))


Training Style Classifier
Training done, accuracy is :  0.8916666666666667
Training done, f1-score is :  0.8916591429960413
              precision    recall  f1-score   support

           0     0.8738    0.9122    0.8926       296
           1     0.9107    0.8717    0.8908       304

    accuracy                         0.8917       600
   macro avg     0.8922    0.8919    0.8917       600
weighted avg     0.8925    0.8917    0.8916       600

confusion matrix:  [0.91216216 0.87171053]
Accuracy: 0.8916666666666667
Recall:  0.8919363442389758
Precision:  0.8922196643645948
F1: 0.8916591429960413
None
[0.08491708 0.07241172 0.         0.         0.03776766 0.02243011
 0.04445282 0.02264715 0.02326764 0.02581475 0.0207359  0.0485558
 0.01982456 0.05701068 0.02718031 0.03009145 0.0258281  0.0237847
 0.01561284 0.03334435 0.04642097 0.02548325 0.01993818 0.0238193
 0.01594941 0.03800167 0.03089883 0.02286418 0.02796249 0.05859108
 0.         0.         0.         0.         0.         0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Training done, accuracy is :  0.8916666666666667
Training done, f1-score is :  0.8916591429960413
              precision    recall  f1-score   support

           0     0.8738    0.9122    0.8926       296
           1     0.9107    0.8717    0.8908       304

    accuracy                         0.8917       600
   macro avg     0.8922    0.8919    0.8917       600
weighted avg     0.8925    0.8917    0.8916       600

confusion matrix:  [0.91216216 0.87171053]
Accuracy: 0.8916666666666667
Recall:  0.8919363442389758
Precision:  0.8922196643645948
F1: 0.8916591429960413
None
[0. 0. 1. 0.]

Char N-gram
Training done, accuracy is :  0.9133333333333333
Training done, f1-score is :  0.9132552630700964
              precision    recall  f1-score   support

           0     0.9266    0.8953    0.9107       296
           1     0.9013    0.9309    0.9159       304

    accuracy                         0.9133       600
   macro avg     0.9139    0.9131    0.9133       600
weighted avg     0.

In [128]:
#random forest

In [129]:
#Training Style Classifier
print("Training Style Classifier")

X_style_train = nlp_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB",  ]]
X_style_test = nlp_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB", ]]

clf = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(X_style_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_style_test)
y_proba = clf.predict_proba(X_style_test)
y_proba_train = clf.predict_proba(X_style_train)
score_style = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_style)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nTraining pos Classifier")

#Training pos Classifier

X_pos_train = nlp_train[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]
X_pos_test = nlp_test[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]

clf = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(X_pos_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_pos_test)
y_proba_pos = clf.predict_proba(X_pos_test)
y_proba_train_pos = clf.predict_proba(X_pos_train)
score_pos = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_pos)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nModel Combination")

feat_for_BERT_LR_train = np.concatenate([raw_out_train, y_proba_train], axis=1)
feat_for_BERT_LR_test = np.concatenate([raw_outputs, y_proba], axis=1)

# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['label'])
clf = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(feat_for_BERT_LR_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_LR_test)
score_comb = accuracy_score(nlp_test['label'], y_pred)
f1_comb = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_comb)
print("Training done, f1-score is : ", f1_comb)
confusion_matrix(nlp_test['label'], y_pred)

# plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nChar N-gram")

#char n-gram
feats_train = nlp_train['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values
feats_test = nlp_test['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values

feats_train = pd.DataFrame(feats_train)[0].apply(lambda x: pd.Series(x))
feats_test = pd.DataFrame(feats_test)[0].apply(lambda x: pd.Series(x))

clf_char = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(feats_train, nlp_train['label'])
# clf_char = LogisticRegression(random_state=0).fit(feats_train, nlp_train['label'])
y_pred = clf_char.predict(feats_test)
y_proba = clf_char.predict_proba(feats_test)
y_proba_train = clf_char.predict_proba(feats_train)

score_char = accuracy_score(nlp_test['label'], y_pred)
f1_char = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_char)
print("Training done, f1-score is : ", f1_char)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nfEATURES FROM bert & Stylometric")

feat_for_BERT_full_train = np.concatenate([feat_for_BERT_LR_train, y_proba_train], axis=1)
feat_for_BERT_full_test = np.concatenate([feat_for_BERT_LR_test, y_proba], axis=1)

clf = RandomForestClassifier(random_state=42,n_estimators=150,n_jobs=-1).fit(feat_for_BERT_full_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_full_test)
score_comb_fin = accuracy_score(nlp_test['label'], y_pred)
f1_comb_fin = f1_score(nlp_test['label'], y_pred, average="macro")
print("Training done, accuracy is : ", score_comb_fin)
print("Training done, f1-score is : ", f1_comb_fin)
confusion_matrix(nlp_test['label'], y_pred)

list_scores, list_f1 = [] , []
list_scores.append([score_lr, score_bert, score_style, score_comb])
list_f1.append([f1_lr, f1_bert, f1_style, f1_comb])

print(get_classification_report(y_test, y_pred, 2))


Training Style Classifier
Training done, accuracy is :  0.885
Training done, f1-score is :  0.884961334226226
              precision    recall  f1-score   support

           0     0.8874    0.8784    0.8829       296
           1     0.8827    0.8914    0.8871       304

    accuracy                         0.8850       600
   macro avg     0.8851    0.8849    0.8850       600
weighted avg     0.8850    0.8850    0.8850       600

confusion matrix:  [0.87837838 0.89144737]
Accuracy: 0.885
Recall:  0.8849128733997156
Precision:  0.8850540850018342
F1: 0.884961334226226
None

Training pos Classifier
Training done, accuracy is :  0.5066666666666667
Training done, f1-score is :  0.33628318584070793
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       296
           1     0.5067    1.0000    0.6726       304

    accuracy                         0.5067       600
   macro avg     0.2533    0.5000    0.3363       600
weighted avg     0.256

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Training done, accuracy is :  0.885
Training done, f1-score is :  0.8848845647997019
              precision    recall  f1-score   support

           0     0.8982    0.8649    0.8812       296
           1     0.8730    0.9046    0.8885       304

    accuracy                         0.8850       600
   macro avg     0.8856    0.8847    0.8849       600
weighted avg     0.8855    0.8850    0.8849       600

confusion matrix:  [0.86486486 0.90460526]
Accuracy: 0.885
Recall:  0.8847350640113798
Precision:  0.8856307435254804
F1: 0.8848845647997019
None

Char N-gram
Training done, accuracy is :  0.915
Training done, f1-score is :  0.9149468417761102
              precision    recall  f1-score   support

           0     0.9239    0.9020    0.9128       296
           1     0.9068    0.9276    0.9171       304

    accuracy                         0.9150       600
   macro avg     0.9153    0.9148    0.9149       600
weighted avg     0.9152    0.9150    0.9150       600

confusion matrix:

In [69]:
#Logistic Regression Classifier

In [130]:
#Training Style Classifier
print("Training Style Classifier")

X_style_train = nlp_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB",  ]]
X_style_test = nlp_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB", ]]

clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_style_test)
y_proba = clf.predict_proba(X_style_test)
y_proba_train = clf.predict_proba(X_style_train)
score_style = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_style)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nTraining pos Classifier")

#Training pos Classifier

X_pos_train = nlp_train[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]
X_pos_test = nlp_test[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]

clf = LogisticRegression(random_state=0).fit(X_pos_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_pos_test)
y_proba_pos = clf.predict_proba(X_pos_test)
y_proba_train_pos = clf.predict_proba(X_pos_train)
score_pos = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_pos)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nModel Combination")

feat_for_BERT_LR_train = np.concatenate([raw_out_train, y_proba_train], axis=1)
feat_for_BERT_LR_test = np.concatenate([raw_outputs, y_proba], axis=1)

# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['label'])
clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_LR_test)
score_comb = accuracy_score(nlp_test['label'], y_pred)
f1_comb = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_comb)
print("Training done, f1-score is : ", f1_comb)
confusion_matrix(nlp_test['label'], y_pred)

# plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nChar N-gram")

#char n-gram
feats_train = nlp_train['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values
feats_test = nlp_test['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values

feats_train = pd.DataFrame(feats_train)[0].apply(lambda x: pd.Series(x))
feats_test = pd.DataFrame(feats_test)[0].apply(lambda x: pd.Series(x))

clf_char = LogisticRegression(random_state=0).fit(feats_train, nlp_train['label'])
# clf_char = LogisticRegression(random_state=0).fit(feats_train, nlp_train['label'])
y_pred = clf_char.predict(feats_test)
y_proba = clf_char.predict_proba(feats_test)
y_proba_train = clf_char.predict_proba(feats_train)

score_char = accuracy_score(nlp_test['label'], y_pred)
f1_char = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_char)
print("Training done, f1-score is : ", f1_char)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nfEATURES FROM bert & Stylometric")

feat_for_BERT_full_train = np.concatenate([feat_for_BERT_LR_train, y_proba_train], axis=1)
feat_for_BERT_full_test = np.concatenate([feat_for_BERT_LR_test, y_proba], axis=1)

clf = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_full_test)
score_comb_fin = accuracy_score(nlp_test['label'], y_pred)
f1_comb_fin = f1_score(nlp_test['label'], y_pred, average="macro")
print("Training done, accuracy is : ", score_comb_fin)
print("Training done, f1-score is : ", f1_comb_fin)
confusion_matrix(nlp_test['label'], y_pred)

list_scores, list_f1 = [] , []
list_scores.append([score_lr, score_bert, score_style, score_comb])
list_f1.append([f1_lr, f1_bert, f1_style, f1_comb])

print(get_classification_report(y_test, y_pred, 2))


Training Style Classifier
Training done, accuracy is :  0.6216666666666667
Training done, f1-score is :  0.6214889767696501
              precision    recall  f1-score   support

           0     0.6088    0.6520    0.6297       296
           1     0.6360    0.5921    0.6133       304

    accuracy                         0.6217       600
   macro avg     0.6224    0.6221    0.6215       600
weighted avg     0.6226    0.6217    0.6214       600

confusion matrix:  [0.65202703 0.59210526]
Accuracy: 0.6216666666666667
Recall:  0.6220661450924608
Precision:  0.6224376051989164
F1: 0.6214889767696501
None

Training pos Classifier
Training done, accuracy is :  0.5066666666666667
Training done, f1-score is :  0.33628318584070793
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       296
           1     0.5067    1.0000    0.6726       304

    accuracy                         0.5067       600
   macro avg     0.2533    0.5000    0.3363     

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Training done, accuracy is :  0.6633333333333333
Training done, f1-score is :  0.6566027471780225
              precision    recall  f1-score   support

           0     0.7136    0.5304    0.6085       296
           1     0.6342    0.7928    0.7047       304

    accuracy                         0.6633       600
   macro avg     0.6739    0.6616    0.6566       600
weighted avg     0.6734    0.6633    0.6572       600

confusion matrix:  [0.53040541 0.79276316]
Accuracy: 0.6633333333333333
Recall:  0.6615842816500711
Precision:  0.6739234449760765
F1: 0.6566027471780225
None

fEATURES FROM bert & Stylometric
Training done, accuracy is :  0.9916666666666667
Training done, f1-score is :  0.991666458328125
              precision    recall  f1-score   support

           0     0.9834    1.0000    0.9916       296
           1     1.0000    0.9836    0.9917       304

    accuracy                         0.9917       600
   macro avg     0.9917    0.9918    0.9917       600
weighted avg 

In [133]:
#Training Style Classifier
print("Training Style Classifier")

X_style_train = nlp_train[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB",  ]]
X_style_test = nlp_test[["avg_len", "num_short_w", "per_digit", "per_cap", "f_a", "f_b", "f_c", "f_d", "f_e", "f_f", "f_g", "f_h", "f_i", "f_j", "f_k", "f_l", "f_m", "f_n", "f_o", "f_p", "f_q", "f_r", "f_s", "f_t", "f_u", "f_v", "f_w", "f_x", "f_y", "f_z", "f_0", "f_1", "f_2", "f_3", "f_4", "f_5", "f_6", "f_7", "f_8", "f_9", "f_e_0", "f_e_1", "f_e_2", "f_e_3", "f_e_4", "f_e_5", "f_e_6", "f_e_7", "f_e_8", "f_e_9", "f_e_10", "f_e_11", "richness",]]# "CC", "CD", "DT", "EX", "FW", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NN", "NNS", "NNP", "PDT", "POS", "PRP", "PRPs", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WPs", "WRB", ]]

clf = BernoulliNB().fit(X_style_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_style_test)
y_proba = clf.predict_proba(X_style_test)
y_proba_train = clf.predict_proba(X_style_train)
score_style = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_style)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nTraining pos Classifier")

#Training pos Classifier

X_pos_train = nlp_train[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]
X_pos_test = nlp_test[['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP', 'PDT', 'POS', 'PRP', 'PRPs', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WPs', 'WRB']]

clf = BernoulliNB().fit(X_pos_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(X_style_train, nlp_train['label'])
y_pred = clf.predict(X_pos_test)
y_proba_pos = clf.predict_proba(X_pos_test)
y_proba_train_pos = clf.predict_proba(X_pos_train)
score_pos = accuracy_score(nlp_test['label'], y_pred)
f1_style = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_pos)
print("Training done, f1-score is : ", f1_style)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nModel Combination")

feat_for_BERT_LR_train = np.concatenate([raw_out_train, y_proba_train], axis=1)
feat_for_BERT_LR_test = np.concatenate([raw_outputs, y_proba], axis=1)

# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_LR_train, nlp_train['label'])
clf = BernoulliNB().fit(feat_for_BERT_LR_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_LR_test)
score_comb = accuracy_score(nlp_test['label'], y_pred)
f1_comb = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_comb)
print("Training done, f1-score is : ", f1_comb)
confusion_matrix(nlp_test['label'], y_pred)

# plot_confusion_matrix_from_data(nlp_test['Target'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nChar N-gram")

#char n-gram
feats_train = nlp_train['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values
feats_test = nlp_test['text'].apply(lambda x: find_freq_n_gram_in_txt(x, list_bigram, list_trigram)).values

feats_train = pd.DataFrame(feats_train)[0].apply(lambda x: pd.Series(x))
feats_test = pd.DataFrame(feats_test)[0].apply(lambda x: pd.Series(x))

clf_char = BernoulliNB().fit(feats_train, nlp_train['label'])
# clf_char = LogisticRegression(random_state=0).fit(feats_train, nlp_train['label'])
y_pred = clf_char.predict(feats_test)
y_proba = clf_char.predict_proba(feats_test)
y_proba_train = clf_char.predict_proba(feats_train)

score_char = accuracy_score(nlp_test['label'], y_pred)
f1_char = f1_score(nlp_test['label'], y_pred, average="macro")

print("Training done, accuracy is : ", score_char)
print("Training done, f1-score is : ", f1_char)
confusion_matrix(nlp_test['label'], y_pred)

print(get_classification_report(y_test, y_pred, 2))

print("\nfEATURES FROM bert & Stylometric")

feat_for_BERT_full_train = np.concatenate([feat_for_BERT_LR_train, y_proba_train], axis=1)
feat_for_BERT_full_test = np.concatenate([feat_for_BERT_LR_test, y_proba], axis=1)

clf = BernoulliNB().fit(feat_for_BERT_full_train, nlp_train['label'])
# clf = LogisticRegression(random_state=0).fit(feat_for_BERT_full_train, nlp_train['label'])

y_pred = clf.predict(feat_for_BERT_full_test)
score_comb_fin = accuracy_score(nlp_test['label'], y_pred)
f1_comb_fin = f1_score(nlp_test['label'], y_pred, average="macro")
print("Training done, accuracy is : ", score_comb_fin)
print("Training done, f1-score is : ", f1_comb_fin)
confusion_matrix(nlp_test['label'], y_pred)

list_scores, list_f1 = [] , []
list_scores.append([score_lr, score_bert, score_style, score_comb])
list_f1.append([f1_lr, f1_bert, f1_style, f1_comb])

print(get_classification_report(y_test, y_pred, 2))


Training Style Classifier
Training done, accuracy is :  0.595
Training done, f1-score is :  0.5948638292314917
              precision    recall  f1-score   support

           0     0.5904    0.5845    0.5874       296
           1     0.5993    0.6053    0.6023       304

    accuracy                         0.5950       600
   macro avg     0.5949    0.5949    0.5949       600
weighted avg     0.5950    0.5950    0.5950       600

confusion matrix:  [0.58445946 0.60526316]
Accuracy: 0.595
Recall:  0.5948613086770982
Precision:  0.5948961101043901
F1: 0.5948638292314917
None

Training pos Classifier
Training done, accuracy is :  0.5066666666666667
Training done, f1-score is :  0.33628318584070793
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000       296
           1     0.5067    1.0000    0.6726       304

    accuracy                         0.5067       600
   macro avg     0.2533    0.5000    0.3363       600
weighted avg     0.2

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

Training done, accuracy is :  0.6116666666666667
Training done, f1-score is :  0.6113546708329742
              precision    recall  f1-score   support

           0     0.5981    0.6486    0.6224       296
           1     0.6272    0.5757    0.6003       304

    accuracy                         0.6117       600
   macro avg     0.6127    0.6122    0.6114       600
weighted avg     0.6129    0.6117    0.6112       600

confusion matrix:  [0.64864865 0.57565789]
Accuracy: 0.6116666666666667
Recall:  0.6121532716927454
Precision:  0.6126854922453355
F1: 0.6113546708329742
None

fEATURES FROM bert & Stylometric
Training done, accuracy is :  0.99
Training done, f1-score is :  0.9899995555358017
              precision    recall  f1-score   support

           0     0.9833    0.9966    0.9899       296
           1     0.9967    0.9836    0.9901       304

    accuracy                         0.9900       600
   macro avg     0.9900    0.9901    0.9900       600
weighted avg     0.9901   

AttributeError: ignored